In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# set up database
conn = sqlite3.connect("student_grades.db")
cursor = conn.cursor()

# create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS students (
               student_id INTEGER PRIMARY KEY AUTOINCREMENT,
               first_name TEXT,
               last_name TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS grades (
               grade_id INTEGER PRIMARY KEY AUTOINCREMENT,
               student_id INTEGER,
               subject TEXT,
               grade INTEGER,
               FOREIGN KEY (student_id) REFERENCES students(student_id)
)
''')
conn.commit()
conn.close()

OperationalError: unable to open database file

In [5]:
# insert data
conn = sqlite3.connect("student_grades.db")
students = [
    ("Alice", "Johnson"), 
    ("Bob", "Smith"), 
    ("Carol", "White"), 
    ("David", "Brown"), 
    ("Eve", "Davis")
    ]

grades = [
    (1, 'Math', 95), (1, 'English', 88), (1, 'History', 90),
    (2, 'Math', 82), (2, 'English', 76), (2, 'History', 85),
    (3, 'Math', 99), (3, 'English', 96), (3, 'History', 97),
    (4, 'Math', 70), (4, 'English', 72), (4, 'History', 62),
    (5, 'Math', 91), (5, 'English', 86), (5, 'History', 88)
]

cursor.executemany("INSERT INTO students (first_name, last_name) VALUES (?, ?)", students)
cursor.executemany("INSERT INTO grades (student_id, subject, grade) VALUES (?, ?, ?)", grades)

conn.commit()
conn.close()

OperationalError: unable to open database file

In [ ]:
# excecute queries
conn = sqlite3.connect("student_grades.db")

cursor.execute('''
               SELECT students.first_name, students.last_name, grades.subject, grades.grade
               FROM students
               JOIN grades ON students.student_id = grades.student_id
''')

cursor.execute('''
               SELECT students.first_name, students.last_name, AVG(grades.grade)
               FROM students
               JOIN grades ON students.student_id = grades.student_id
               GROUP BY students.student_id
''')

cursor.execute('''
               SELECT students.first_name, students.last_name
               FROM students
               JOIN grades ON students.student_id = grades.student_id
               GROUP BY students.student_id
               ORDER BY AVG(grades.grade) DESC
               LIMIT 1
''')

cursor.execute('''
               SELECT AVG(grade)
               FROM grades
               WHERE subject = "Math"
               GROUP BY subject
''')

cursor.execute('''
               SELECT DISTINCT students.first_name, students.last_name
               FROM students
               JOIN grades ON students.student_id = grades.student_id 
               WHERE grades.grade > 90

''')
conn.close()

In [ ]:
# load data into pandas
conn = sqlite3.connect("student_grades.db")

df_students = pd.read_sql_query("SELECT * FROM students", conn)
df_grades = pd.read_sql_query("SELECT * FROM grades", conn)


print(df_students)
print(df_grades)

In [ ]:
# join to make a single dataframe

df_combined = pd.read_sql_query('''
                                SELECT students.first_name, students.last_name, grades.subject, grades.grade
                                FROM students
                                JOIN grades ON students.student_id = grades.student_id
                                ''', conn)

print(df_combined)

In [ ]:
## graphs ##

# average grade per student
df_combined['full_name'] = df_combined['first_name'] + ' ' + df_combined['last_name']
avg_grades_per_student = df_combined.groupby('full_name')['grade'].mean()

# plot
avg_grades_per_student.plot(kind="bar", title="Average Grades per Student")
plt.ylabel('Average Grade')
plt.xlabel('Student')
plt.tight_layout()
plt.show()


# average grade per student
avg_grades_per_subject = df_combined.groupby('subject')['grade'].mean()

# graph
avg_grades_per_subject.plot(kind='bar', title='Average Grades per Subject')
plt.ylabel('Average Grade')
plt.xlabel('Subject')
plt.tight_layout()
plt.show()

In [ ]:
# bonus!!

# query students with the highest grade
conn = sqlite3.connect("student_grades.db")
df_highest_grade_per_subject = pd.read_sql_query('''
                                                SELECT students.first_name, students.last_name, grades.subject, MAX(grades.grade) as max_grade
                                                FROM students
                                                JOIN grades ON students.student_id = grades.student_id
                                                GROUP BY grades.subject 
                                                ORDER BY max_grade DESC
                                                ''', conn)


# first and last names
df_highest_grade_per_subject['full_name'] = df_highest_grade_per_subject['first_name'] + ' ' + df_highest_grade_per_subject['last_name']

# graph
df_highest_grade_per_subject.plot(x='subject', y='max_grade', kind='bar', title='Highest Grade in Each Subject')
plt.ylabel('Max Grade')
plt.xlabel('Subject')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# close connection
conn.close()